<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Piping-(Postfix)-Notation-in-Python" data-toc-modified-id="Piping-(Postfix)-Notation-in-Python-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Piping (Postfix) Notation in Python</a></span><ul class="toc-item"><li><span><a href="#Notebook's-Purpose" data-toc-modified-id="Notebook's-Purpose-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Notebook's Purpose</a></span></li><li><span><a href="#Approaches" data-toc-modified-id="Approaches-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Approaches</a></span></li><li><span><a href="#Overwriting-Operators" data-toc-modified-id="Overwriting-Operators-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Overwriting Operators</a></span></li><li><span><a href="#Pipe()-Function" data-toc-modified-id="Pipe()-Function-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span><code>Pipe()</code> Function</a></span></li></ul></li><li><span><a href="#Original-Source" data-toc-modified-id="Original-Source-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Original Source</a></span></li></ul></div>

# Piping (Postfix) Notation in Python

## Notebook's Purpose

The purpose of this notebook is to document a simple, robust implementation of Linix-style "piping" in Python.  There are ways to overwrite operators in Python, but they seem overly complication given the goals of piping.

The main purposes of piping is to daisy-chain methods or transformations, which is very common in functional programming and data science.  Imagine writing code like this

    func1(func2(func3(func4(func5(func5(x))))))

Needless to say, this is ideal for a computer.  However, it is suboptimal in terms of human readability.  It would be far easier to read (and equally easy for a computer to parse) written as follows

    func5(x) | func4 | func3 | func2 | func1

## Approaches

I present two ways to add postfix notation to Python:

   1. Overloading an Operator (cannot recall where in the Internet I learned this)
   2. Imitating /Mathematica/'s `//` operator, which works exactly as pipes in Linux.  In fact, I am actually implementing a version of _Mathematica_'s `Composition[]` operator.  See https://reference.wolfram.com/language/ref/Composition.html.

## Overwriting Operators

Let us define a class called `Postfix` that overloads the right-shift operator (e.g. `a | b`).  The sole purpose of the class is to turn `| Postfix(func)` into a postfix operator.

In [2]:
class Postfix:
    def __init__(self, f):
        self.f = f

    # Overload the right-shift operator (which is invoked as a|b)
    def __ror__(self, other):
        return self.f(other)

Let us consider a silly example:

In [3]:
[1, 2, 3] | Postfix(sum)

6

Let us consider another simple example to square every element in a list.

In [5]:
[1, 2, 3] | Postfix(lambda x: [x**2 for x in x])

[1, 4, 9]

Of course, the example is silly since we could have written

    [x**2 for x in [1, 2, 3]]

However, the point of the example is to illustrate the notation.

Let us consider a more complicated example.  Start from the matrix:

    [[  1,   2,    3],
     [ 10,  20,   30],
     [100, 200,  300]
     ]

 1. Sum each row in the above matrix
 2. Multiply totals

In [7]:
def getMatrixRowTotals(aMatrix):
    """Return list with matrix row totals."""
    return list(map(lambda x: sum(x),
                    aMatrix
                   )
               )

def reduceListUsingProduct(aList):
    """Return total from multiplying elements in list."""
    result = 1
    for x in aList:
        result *= x
    
    return result

# Get total of each row, followed by taking product of totals
[[  1,   2,   3],
 [ 10,  20,  30],
 [100, 200, 300]
 ]\
|Postfix(getMatrixRowTotals)\
|Postfix(reduceListUsingProduct)

216000

The above example is simple, but it is easy to imagine an arbitrarily long chain of complicated operators.  In my opinion, this is much easier to read than standard composition of functions.

## `Pipe()` Function

Let us now introduce a helper function, and repeat the last example above using `Pipe()`.

In [9]:
def Pipe(data, *operators):
    """Daisychain functions in`operators` starting from `data`."""
    if len(operators)==0:
        # Return data if no operators passed
        return data
    elif len(operators)==1:
        # Return data evaluated ny first operator when only one passed
        return operators[0](data)
    else:
        # Evaluate data and recurse when more than one operator passed
        return Pipe(operators[0](data), *operators[1:])

For example, assuming `f1`, `f2`, and `f3` are functions, you can invoke `Pipe()` as follows:

    Pipe(x, f1, f2, f3)

This will compute `f3(f2(f1(x)))`.

In [10]:
Pipe([[  1,   2,   3],
      [ 10,  20,  30],
      [100, 200, 300]
     ],
     getMatrixRowTotals,
     reduceListUsingProduct
    )

216000

As you can see, we got the same reasult as with `|Postfix`.  In any case, I prefer `Pipe` over `|Postfix`.

# Original Source

CodeSamples/PythonProgrammingSamples/Syntax/PipingAndPostFix.ipynb